In [13]:
import gzip
from pathlib import Path
import shutil

RUNID = "lxr-instrumentation-bee-2024-06-08-Sat-053011"

LOGDIR = Path("/home/wenyuz/MMTk-Dev/evaluation/results/log") / RUNID

LOGFILES = list(LOGDIR.glob("*.jdk-lxr-objsize.*.log.gz"))

BENCHMARKS = list(set(f.name.split(".")[0] for f in LOGFILES))

# print(BENCHMARKS, len(BENCHMARKS))


def parse_log_file(log: Path):
    bench = log.name.split(".")[0]
    data_lines = []
    recording = False
    recording_objsize = False
    with gzip.open(log, "rt") as f:
        for line in f:
            if f"===== DaCapo 23.11-chopin {bench} starting =====" in line:
                recording = True
            elif f"===== DaCapo 23.11-chopin {bench}" in line:
                recording = False
            elif recording:
                if "Static Size Distribution:" in line:
                    recording_objsize = True
                    data_lines = []
                elif recording_objsize:
                    if not line.startswith(" - obj-size="):
                        recording_objsize = False
                    else:
                        data_lines.append(line.strip())
    if len(data_lines) > 0:
        # print(data_lines)
        dist = []
        for line in data_lines:
            segs = line.split()
            size_class = int(segs[1].split("=")[1])
            count = int(segs[2].split("=")[1])
            total_size = int(segs[3].split("=")[1])
            dist.append((size_class, count, total_size))
        # print(stats_list)
        return dist
    return False


count = 0
bms = []
for logfile in LOGFILES:
    # print("🟢 " + logfile.name)
    benchmark = logfile.name.split(".")[0]
    # print("🟣 " + benchmark)
    if parse_log_file(logfile):
        count += 1
        bms.append(benchmark)

    # break
print(count)
for b in BENCHMARKS:
    if b not in bms:
        print(b)
# print(set(bms) - set(BENCHMARKS))

# with gzip.open(LOGDIR / "avrora.1989.10.jdk-lxr-alloc.lxr.common3.tph.mmtk_verbose.dacapochopin.log.gz", "rt") as f_in:
#     for line in f_in:
#         print(line)

21
kafka
